# The Git Paradigm

## The Disaster That Created Git (History)

**The Story:** In 2005, the Linux kernel (millions of lines of code) used a proprietary VCS called **BitKeeper**. When the free license was revoked, Linus Torvalds (creator of Linux) decided to build his own tool in two weeks. He had specific requirements:

- **Distributed:** Everyone has the full history (no single point of failure)
- **Fast:** Handle the Linux kernel instantly  
- **Safe:** Cryptographic integrity (impossible to corrupt history without detection)
- **Simple branches:** Branching should take less than a second

**Reference:** [Git SCM - A Short History of Git](https://git-scm.com/book/en/v2/Getting-Started-A-Short-History-of-Git)


## Installing Git
- Install Git ([Download](https://git-scm.com/downloads))
- Check if installed
    - `git --version`


## First Steps: Identity & Repository

```bash
# Create your project
mkdir python_project
cd python_project
git init

# Who are you? (Stored in every commit forever)
git config --global user.name "Your Name"
git config --global user.email "your.email@example.com"

# Check settings
git config --list

# Set default branch name (modern Git)
git config --global init.defaultBranch main
```

## Inside the .git Directory

When you run `git init`, Git creates a `.git` directory. This IS your repository—everything else is just the current checkout.

```bash
.git/
├── HEAD              # Points to current branch (e.g., "ref: refs/heads/main")
├── config            # Repository-specific settings
├── description       # For GitWeb (unused usually)
├── hooks/            # Scripts that run before/after commands
├── info/             # Additional repo info
├── objects/          # THE DATABASE (compressed snapshots)
│   ├── 00/
│   ├── 01/
│   └── ...           # 256 folders (first 2 chars of SHA-1)
├── refs/             # Pointers to commits
│   ├── heads/        # Local branches (main, feature-x)
│   ├── tags/         # Tags (v1.0, release)
│   └── remotes/      # Remote branch caches
└── index             # The Staging Area (binary file)
```

### Exploring Objects
```bash
# Create a file and commit it
echo "hello" > test.txt
git add test.txt
git commit -m "Test"

# See the object storage
ls .git/objects/
# You'll see folders like "ce", "d8", etc.

# Look at a specific object
git cat-file -p d8d8d8...  # View content
git cat-file -t d8d8d8...  # View type (blob, tree, commit)
```

**Types of Objects:**
- **Blob:** File content (compressed)
- **Tree:** Directory listing (filenames + blob hashes)
- **Commit:** Author, message, timestamp, parent commit hash, tree hash
- **Tag:** Named reference to a commit

## The Three States (The Git Philosophy)

Git has three distinct boxes where your code lives. This is unique to Git and confuses beginners, but it's the source of its power.

### The Three Boxes:

1. **Working Directory (Sandbox)**
   - Your actual files on disk
   - You can break things here freely
   - Git doesn't track this until you tell it to

2. **Staging Area / Index (The Lobby)**
   - A proposed next commit (the `.git/index` file)
   - Like a "loading dock" where you prepare packages
   - Allows you to craft perfect commits from partial changes

3. **Repository (The Vault)**
   - The permanent history (the `.git/objects` folder)
   - Once committed, it's there forever (almost)

**The Flow:**
```
Working Dir → git add → Staging Area → git commit → Repository
```

### Why Three States?
Linus designed this so you can:
- **Work messily** (Working Dir): Write experimental code
- **Curate carefully** (Staging): Select exactly what belongs in the commit  
- **Preserve permanently** (Repository): Immutable history

## The First Commit Chain

```bash
# Create a file
echo "print('Hello World')" > hello.py

# Check status
git status
# "Untracked files" = Working Directory only

# Move to Staging
git add hello.py

# Check status
git status  
# "Changes to be committed" = Staging Area

# Commit to Repository
git commit -m "Initial commit: Add hello script"
```

### The Editor Question: What if you don't use -m?

If you run `git commit` without `-m`, Git opens your default text editor. But how does Git know which editor?

**The Priority Order:**
1. `GIT_EDITOR` environment variable
2. `core.editor` Git configuration
3. `VISUAL` environment variable  
4. `EDITOR` environment variable
5. System default (usually Vim or Nano)

**Setting your editor:**
```bash
# VS Code
git config --global core.editor "code --wait"

# Vim (default on most systems)
git config --global core.editor vim

# Nano (beginner-friendly)
git config --global core.editor nano
```

When the editor opens, you write your commit message on the first line, save, and close. If you leave the first line empty, Git aborts the commit.

### Golden Rules
- A commit should be one logical unit of work. Not "Tuesday afternoon's random changes."
- Write good commit message. Read [this article](https://cbea.ms/git-commit/)

## Recovering Untracked Files (The "Oops I Deleted It" Scenario)

**The Problem:** You create a new file, never `git add` it, never commit it, then accidentally delete it. It's not in Git's history!

**Prevention (Git Clean):**
```bash
# See what untracked files exist
git status

# Remove untracked files (DANGER)
git clean -n  # Dry run (see what would be deleted)
git clean -f  # Actually delete untracked files
git clean -fd # Delete untracked files AND directories
```

**Recovery Options:**
1. **Trash/Recycle Bin:** Check your OS trash first
2. **IDE Undo:** If you deleted in an IDE, use Ctrl+Z
3. **File Recovery Tools:** PhotoRec, TestDisk (last resort)

**Lesson:** If a file is important, `git add` it immediately, even if not ready to commit. Or use `git stash -u` to stash untracked files.

## Understanding History

```bash
# The default log (verbose)
git log

# The useful log (one line, graph, decorations)
git log --oneline --graph --decorate --all

# See what changed in the last commit
git show HEAD

# See what you haven't committed yet
git diff              # Working vs Staging
git diff --cached     # Staging vs Repository (what will be committed)
```

### The HEAD Pointer
`HEAD` is a special pointer: **"You are here"**

```bash
cat .git/HEAD
# Output: ref: refs/heads/main

cat .git/refs/heads/main
# Output: a1b2c3d... (the hash of current commit)
```

When you checkout an old commit, HEAD moves there. When you commit, HEAD moves forward.

## Python .gitignore (Standard)

Create this immediately:

```gitignore
# Byte-compiled / optimized / DLL files
__pycache__/
*.py[cod]
*$py.class
*.so
.Python

# Distribution / packaging
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
*.egg-info/
.installed.cfg
*.egg

# Virtual environments
venv/
ENV/
env/
.venv/
env.bak/
venv.bak/

# IDE
.idea/
.vscode/
*.swp
*.swo
*~

# OS
.DS_Store
Thumbs.db

# Testing
.pytest_cache/
.coverage
htmlcov/
.tox/

# Jupyter
.ipynb_checkpoints/

# Environment variables
.env
```